<a href="https://colab.research.google.com/github/Jason20262/-SeminarioBigdata/blob/main/Lab_5_analisis_sentimientos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install pandas openpyxl textblob nltk pysentimiento

In [14]:
# Importación de las librerías
import pandas as pd
from textblob import TextBlob
import nltk
import re
from nltk.corpus import stopwords
from pysentimiento import create_analyzer
import transformers

nltk.download('punkt')
nltk.download('stopwords')

transformers.logging.set_verbosity(transformers.logging.ERROR)

analyzer = create_analyzer(task="sentiment", lang="es")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
analyzer.predict("Asi es vivir")

AnalyzerOutput(output=NEU, probas={NEU: 0.584, POS: 0.242, NEG: 0.174})

In [16]:
def limpiar_texto(texto):
    if isinstance(texto, float):
        texto = ""
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar URLs
    texto = re.sub(r'http\S+', '', texto)
    # Eliminar menciones a usuarios de Twitter
    texto = re.sub(r'@\w+', '', texto)
    # Eliminar hashtags
    texto = re.sub(r'#\w+', '', texto)
    # Eliminar signos de puntuación y caracteres especiales
    texto = re.sub(r'[^\w\s]', '', texto)
    # Eliminar palabras vacías (stopwords)
    stop_words = set(stopwords.words('spanish'))
    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stop_words]
    texto = ' '.join(palabras)
    return texto

In [17]:
hate_speech_analyzer = create_analyzer(task="hate_speech", lang="es")
hate_speech_analyzer.predict("odio")

AnalyzerOutput(output=[], probas={hateful: 0.049, targeted: 0.025, aggressive: 0.044})

In [18]:
hate_speech_analyzer.predict("odio")

AnalyzerOutput(output=[], probas={hateful: 0.049, targeted: 0.025, aggressive: 0.044})

In [19]:
def analizar_sentimiento(texto):
    resultado = analyzer.predict(texto)
    return resultado.output, resultado.probas

In [20]:
# Cargue del documento
noticias = pd.read_excel('/content/drive/MyDrive/Noticias2ver.xlsx')

# Aplicar la limpieza de texto a las columnas relevantes
noticias['info_limpio'] = noticias['info'].apply(limpiar_texto)
noticias['contenido_limpio'] = noticias['contenido'].apply(limpiar_texto)

In [21]:
# Aplicar el análisis de sentimiento a las columnas limpias
noticias['sentimiento_info'] = noticias['info_limpio'].apply(lambda x: analizar_sentimiento(x)[0])
noticias['probabilidades_info'] = noticias['info_limpio'].apply(lambda x: analizar_sentimiento(x)[1])
noticias['sentimiento_contenido'] = noticias['contenido_limpio'].apply(lambda x: analizar_sentimiento(x)[0])
noticias['probabilidades_contenido'] = noticias['contenido_limpio'].apply(lambda x: analizar_sentimiento(x)[1])

In [22]:
# Se almacenan resultados en el archivo cargado inicialmente
noticias.to_excel('/content/drive/MyDrive/Noticias2ver.xlsx', index=False)

# Mensaje de confirmación
print("Análisis de sentimiento realizado correctamente, validar el archivo que se cargo inicialmente")

Análisis de sentimiento completado y resultados guardados en Noticias2ver_resultados.xlsx


#Realizado Por:

###Breiner Spencer Barrios Asprilla - 121230
###Johan Sebastian Quintero Vargas - 107699
###Jason Rodriguez - 99229
###Daniel Gutierrez - 90378